<a href="https://colab.research.google.com/github/dkedrowski/ForestConceptsNLP/blob/main/UseCases/UC1-Testing/UC1-CQ2c/UC1_CQ2c_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The purpose of this notebook is to query the SAWGraph knowledge graph for NHD surface water flowlines that are at most 20 km downstream of the flowlines nearest landfill and Department of Defenses (DoD) sites.

This is a demonstration of comepetency question 2c from Use Case 1: Testing.

- What surface water bodies are downstream at most 20 km from landfills or DoD sites?

Note: This query finds flowlines that cross an S2 cell that contains a landfill or a DoD site and then finds flowlines downstream of those flowlines that have a combined length (including the original flowlin) of at most 20 km.

Note: This query currently returns only flowlines; i.e., rivers/streams/etc. It will be possible to intersect the flowlines with ponds/lakes once newer data is added to the Hydrology repository.

# Setup

Here we set up SPARQLWrapper to work with our endpoint and create our query.

## Install & Import Statements

Install: The SPARQLWrapper libary provides tools for querying SPARQL endpoints. The sparql_dataframe library can be used with SPARQLWrapper to convert JSON results from a SPARQL query directly to a Pandas dataframe.

Import: See the inline comments for a brief rationale of each library.

In [1]:
%%capture
!pip install mapclassify --upgrade --quiet
!pip install SPARQLWrapper --upgrade --quiet
!pip install sparql_dataframe --upgrade --quiet

In [2]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For maps
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper, JSON, GET, POST, DIGEST   # For querying SPARQL endpoints
import sparql_dataframe                                            # For converting SPARQL query results to Pandas dataframes

## Variable Initialization

A SPARQLWrapper is created to access the Hydrology repository for the SAWGraph project.

In [3]:
%%capture
pd.options.display.width = 240

endpointGET = 'https://gdb.acg.maine.edu:7200/repositories/Hydrology'

sparqlGET = SPARQLWrapper(endpointGET)
sparqlGET.setHTTPAuth(DIGEST)
sparqlGET.setCredentials('sawgraph-endpoint', 'skailab')
sparqlGET.setMethod(GET)
sparqlGET.setReturnFormat(JSON)

## Query

This query uses federation to access the FIO, S2L13_AdminRegions, and Hydrology repositories.

The first block finds facilities that are either landfills or DoD sites (via the specified NAICS industry codes) and the S2 cells they are within. It pulls label information if it is available as well as geometries.

The second block retrieves the geometries for the S2 cells. This is not necessary to the overall query, but has been added to the map as a way to check the query is functioning properly.

The third block looks for any surface water flowlines that cross the S2 cells that contain landfills or DoD sites. It finds all flowlines downstream of those initial flowlines such that the sum of the length of all connected flowlines is at most 20 km. If available, the name associated with the flowline is retrieved. Geometries are found. All flowlines labeled "Coastline" are excluded from the results.




The query is executed and returned as a dataframe.

In [4]:
%%time
query = """
PREFIX fio: <http://sawgraph.spatialai.org/v1/fio#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX naics: <http://sawgraph.spatialai.org/v1/fio/naics#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX saw_water: <http://sawgraph.spatialai.org/v1/saw_water#>
PREFIX schema: <https://schema.org/>
PREFIX us_frs: <http://sawgraph.spatialai.org/v1/us-frs#>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
    SERVICE <repository:FIO> {
        SELECT * WHERE {
            ?fac rdf:type fio:Facility ;
                 fio:ofIndustry ?code ;
                 kwg-ont:sfWithin ?fac_s2  ;
                 geo:hasGeometry/geo:asWKT ?fac_wkt .
            OPTIONAL { ?fac rdfs:label ?faclabel . }
            OPTIONAL { ?code rdfs:label ?ind . }
            FILTER (?code IN (naics:NAICS-IndustryCode-562212, naics:NAICS-IndustryCode-92811, naics:NAICS-IndustryCode-928110))
        }
    }

    SERVICE <repository:S2L13_AdminRegions> {
        SELECT * WHERE {
            ?fac_s2 rdf:type kwg-ont:S2Cell_Level13 .
            ?fac_s2 geo:hasGeometry/geo:asWKT ?fac_s2_wkt .
        }
    }

    SERVICE <repository:Hydrology> {
        SELECT * WHERE {
            ?fl_us rdf:type hyf:HY_FlowPath ;
            	   kwg-ont:sfCrosses ?fac_s2 ;
            	   hyf:downstreamWaterbodyTC ?fl_ds ;
                   saw_water:hasFTYPE ?fl_us_ftype ;
                   wdp:P2043 ?fl_us_length ;
            	   geo:hasGeometry/geo:asWKT ?fl_us_wkt .
            ?fl_ds wdp:P2043 ?fl_ds_length ;
                   saw_water:hasFTYPE ?fl_ds_ftype ;
            	   geo:hasGeometry/geo:asWKT ?fl_ds_wkt .
           OPTIONAL { ?fl_us schema:name ?fl_us_name . }
           OPTIONAL { ?fl_ds schema:name ?fl_ds_name . }
            {
                SELECT ?fl_ds (SUM(?fl_length) AS ?total_length) WHERE
                {
                    ?fl_us rdf:type hyf:HY_FlowPath ;
                    	   kwg-ont:sfCrosses ?fac_s2 ;
                    	   hyf:downstreamWaterbodyTC ?fl_ds .
                    OPTIONAL {
                        ?fl_us hyf:downstreamWaterbodyTC ?fl .
                        ?fl hyf:downstreamWaterbodyTC ?fl_ds .
                        ?fl wdp:P2043 ?fl_length .
                    }
                } GROUP BY ?fl_ds
            }
            FILTER (?fl_ds_ftype != "Coastline")
            FILTER (?fl_us_ftype != "Coastline")
            FILTER (?total_length < "20.0"^^xsd:float)
        }
    }
}
"""
df = sparql_dataframe.get(endpointGET, query)

CPU times: user 251 ms, sys: 40.9 ms, total: 292 ms
Wall time: 47.4 s


# Visualizing on a map

## Partioning the Query Results

The data is split into four separate themed dataframes, one for the flowlines that cross each facility's S2 cells, one for the downstream flowlines, one for the facilities themselves, and one for the facility S2 cells.

The columns with WKT coordinates in each dataframe are converted to Shapely geometry objects prior to converting the dataframes to GeoDataFrames.

In [5]:
df_fl_us = df[['fl_us', 'fl_us_length', 'fl_us_name', 'fl_us_wkt']].copy()
df_fl_us.drop_duplicates(inplace=True)
df_fl_us['fl_us_wkt'] = df_fl_us['fl_us_wkt'].apply(wkt.loads)

df_fl_ds = df[['fl_us', 'fl_ds', 'fl_ds_length', 'total_length', 'fl_ds_name', 'fl_ds_wkt']].copy()
df_fl_ds.drop_duplicates(inplace=True)
df_fl_ds['fl_ds_wkt'] = df_fl_ds['fl_ds_wkt'].apply(wkt.loads)

df_fac = df[['fac', 'faclabel', 'ind', 'fac_wkt']].copy()
df_fac.drop_duplicates(inplace=True)
df_fac['fac_wkt'] = df_fac['fac_wkt'].apply(wkt.loads)

df_s2 = df[['fac', 'fac_s2', 'faclabel', 'fac_s2_wkt']].copy()
df_s2.drop_duplicates(inplace=True)
df_s2['fac_s2_wkt'] = df_s2['fac_s2_wkt'].apply(wkt.loads)

## Create GeoPandas dataframes for mapping

Convert the above dataframes to GeoDataFrames, setting the WKT columns as the geometry columns and setting the CRS to WGS 84.

In [6]:
%%capture
gdf_fl_us = gpd.GeoDataFrame(df_fl_us, geometry='fl_us_wkt')
gdf_fl_us.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fl_ds = gpd.GeoDataFrame(df_fl_ds, geometry='fl_ds_wkt')
gdf_fl_ds.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fac = gpd.GeoDataFrame(df_fac, geometry='fac_wkt')
gdf_fac.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_s2 = gpd.GeoDataFrame(df_s2, geometry='fac_s2_wkt')
gdf_s2.set_crs(epsg=4326, inplace=True, allow_override=True)

## Create Map

Each GeoDataFrame is a layer in the final map.

In [7]:
%%capture
fl_color = 'blue'
fac_color = 'red'
s2_color = 'black'
boundweight = 5

map = gdf_s2.explore(color=s2_color,
                     style_kwds=dict(weight=boundweight),
                     tooltip=True,
                     name='Facility S2 Cells (black)',
                     show=True)
gdf_fac.explore(m=map,
                color=fac_color,
                style_kwds=dict(weight=boundweight),
                tooltip=True,
                name='Facilities (red)',
                show=True)
gdf_fl_us.explore(m=map,
                  color=fl_color,
                  style_kwds=dict(weight=boundweight),
                  tooltip=True,
                  highlight=False,
                  name='Crossing Flowlines (blue)',
                  show=True)
gdf_fl_ds.explore(m=map,
                  color=fl_color,
                  style_kwds=dict(weight=boundweight),
                  tooltip=True,
                  highlight=False,
                  name='Downstream Flowlines (blue)',
                  show=True)

# folium.TileLayer("stamenterrain", show=False).add_to(map)
# folium.TileLayer("MapQuest Open Aerial", show=False).add_to(map)
folium.LayerControl(collapsed=False).add_to(map)

## Show Map

The map is created inside a Figure box to control its size.

In [8]:
map.save('SAWGraph_UC1_CQ2c_map.html')

fig = Figure(width=800, height=600)
fig.add_child(map)